In [1]:
from PIL import Image
from ImageFolder import ImageFolder

from tqdm import tqdm
import matplotlib.pyplot as plt
from torchsummary import summary

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import CrossEntropyLoss

from torch.optim import SGD
from torch.utils.data import DataLoader

from torchvision import transforms
from torchvision.models import resnet50

import pandas as pd

lr=1e-3

In [2]:
transform = transforms.Compose([
    transforms.CenterCrop((362, 480)),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5]),
    ])


In [3]:
train_data = ImageFolder("/media/mldadmin/home/s123mdg34_04/WangShengyuan/FYP/testdata/data/train", transform)
test_data = ImageFolder("/media/mldadmin/home/s123mdg34_04/WangShengyuan/FYP/testdata/data/test", transform)

train_dataset = DataLoader(train_data, batch_size=256, shuffle=True)
test_dataset = DataLoader(test_data, batch_size=256, shuffle=False)

In [ ]:
class Head(nn.Module):
    def __init__(self, in_features, out_features, Backbone):
        super().__init__()
        self.Backbone = Backbone
        self.linear = nn.Sequential(nn.ReLU(inplace=True),#Reasons for using ReLU: #1 preventing network overfitting #2 non-linear
                                    nn.Linear(in_features=in_features, out_features=out_features,bias=False))
    def forward(self, x):
        x = self.Backbone(x)
        x = self.linear(x)
        x = torch.squeeze(x, 1)
        return x

In [4]:
# model = Head(in_features=1000, out_features=2, Backbone=resnet50)
model = resnet50(pretrained=False)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)

/media/mldadmin/home/s123mdg34_04/anaconda3/envs/FYP/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/media/mldadmin/home/s123mdg34_04/anaconda3/envs/FYP/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [5]:
# Device Settings
device = torch.device('cuda:3' if torch.cuda.is_available() else 'cpu')
model.to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr, momentum=0.9)


df = pd.DataFrame(columns=['epoch', 'train_acc', 'train_loss', 'test_acc', 'test_loss'])
num_epochs = 10


In [6]:
results = []
for epoch in range(num_epochs):
    train_acc = 0
    train_loss = 0
    model.train()
    for inputs, labels, direc in tqdm(train_dataset):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        train_loss += loss.cpu().detach().numpy()
        # train_acc += torch.sum(torch.eq(torch.max(labels, 1).indices, torch.max(outputs, 1).indices).int()).cpu().detach().numpy()
        train_acc += torch.sum(
            torch.eq(
                torch.max(labels, 1).indices,
                torch.max(outputs, 1).indices)).cpu().detach().numpy()
        loss.backward()
        optimizer.step()
    train_loss /= len(train_dataset)
    train_acc = train_acc / len(train_data) * 100
    print('Epoch: {} | Train | Loss: {:.4f}, Acc: {:.2f}'.format(
        epoch, train_loss, train_acc))

    test_acc = 0
    test_loss = 0
    model.eval()
    with torch.no_grad():
        for inputs, labels, direc in tqdm(test_dataset):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss.cpu().detach().numpy()
            # test_acc += torch.sum(torch.eq(torch.max(labels, 1).indices, torch.max(outputs, 1).indices).int()).cpu().detach().numpy()
            test_acc += torch.sum(
                torch.eq(
                    torch.max(labels, 1).indices,
                    torch.max(outputs, 1).indices)).cpu().detach().numpy()
        test_loss /= len(test_dataset)
        test_acc = test_acc / len(test_data) * 100
        print('Epoch: {} | Test | Loss: {:.4f}, Acc: {:.2f}'.format(
            epoch, test_loss, test_acc))
    # add the results to the dataframe

    results.append([epoch, train_acc, train_loss, test_acc, test_loss])
    df1 = pd.DataFrame(results, columns=['epoch', 'train_acc', 'train_loss', 'test_acc', 'test_loss'])
    df = pd.concat([df, df1])
    results.clear()


100%|██████████| 16/16 [00:29<00:00,  1.84s/it]


Epoch: 0 | Train | Loss: 0.5231, Acc: 75.63


100%|██████████| 4/4 [00:07<00:00,  1.78s/it]


Epoch: 0 | Test | Loss: 0.6378, Acc: 69.34


100%|██████████| 16/16 [00:27<00:00,  1.73s/it]


...

Epoch: 8 | Test | Loss: 0.0339, Acc: 99.12


100%|██████████| 16/16 [00:27<00:00,  1.71s/it]


Epoch: 9 | Train | Loss: 0.0151, Acc: 99.54


100%|██████████| 4/4 [00:07<00:00,  1.76s/it]

Epoch: 9 | Test | Loss: 0.0260, Acc: 99.02


In [8]:
# save the dataframe as a csv file
filename = f"Results/results_lr{lr}.csv"
df.to_csv(filename, index=False)
checkpoint_name = f"Checkpoints/checkpoint_lr{lr}.pth"
torch.save(model.state_dict(), checkpoint_name)

In [7]:
df

,epoch,train_acc,train_loss,test_acc,test_loss
0,0,75.630046,0.523058,69.343781,0.637819
0,1,95.816002,0.215310,30.656219,0.861846
0,2,99.217030,0.054081,30.656219,1.752685
0,3,99.265965,0.031479,80.901077,0.379347
0,4,99.339369,0.025708,98.139079,0.067509
0,5,99.363837,0.023059,99.020568,0.033001
0,6,99.437240,0.020853,99.412341,0.026895
0,7,99.437240,0.019263,98.824682,0.037150
0,8,99.486176,0.016851,99.118511,0.033856
0,9,99.535111,0.015125,99.020568,0.025985
